# Gridsearch for best ResNet model 

## Grid search over learning rate, batch sizes and layers (using validation loss)

In [ ]:
def train_and_evaluate_val_loss(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=500, patience=15):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_val_loss = float('inf')  # Initialize the best validation loss as infinity
    best_model_wts = copy.deepcopy(model.state_dict())
    epochs_no_improve = 0
    early_stop = False

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0

        # Training phase
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss / len(train_loader):.4f}, Validation Loss: {avg_val_loss:.4f}')
        
        scheduler.step(avg_val_loss)  # Learning rate scheduler step based on validation loss

        # Check for improvement based on validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Early stopping check
        if epochs_no_improve >= patience:
            print(f'Early stopping triggered after {epoch + 1} epochs.')
            early_stop = True
            break
    
    if not early_stop:
        print('Reached maximum epoch limit.')

    # Load best model weights based on lowest validation loss
    model.load_state_dict(best_model_wts)
    return best_val_loss  # Return the best validation loss achieved


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example hyperparameters for the grid search
batch_sizes = [16, 32]  # Add more as needed
learning_rates = [0.001, 0.0005]  # Add more as needed
unfreeze_options = ['last', 'last_plus_one']  # Options for layers to unfreeze

# Placeholder for best model's performance, now focusing on validation loss
best_loss = float('inf')  # Use infinity as the initial value for the best loss
best_params = {}

# Updated train_and_evaluate function that returns the best validation loss

for batch_size in batch_sizes:
    for lr in learning_rates:
        for unfreeze_option in unfreeze_options:
            print(f"Training with batch size: {batch_size}, learning rate: {lr}, unfreeze option: {unfreeze_option}")

            # Load a pretrained ResNet model
            model = models.resnet18(pretrained=True)
            
            # Freeze all parameters first
            for param in model.parameters():
                param.requires_grad = False
            
            # Unfreeze the selected layers based on unfreeze_option
            if unfreeze_option == 'last':
                for param in model.fc.parameters():
                    param.requires_grad = True
            elif unfreeze_option == 'last_plus_one':
                # Unfreeze both the last block and the fully connected layer
                for param in model.layer4.parameters():
                    param.requires_grad = True
                for param in model.fc.parameters():
                    param.requires_grad = True
            
            model.to(device)

            # Assuming you've already defined your datasets
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

            optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
            criterion = nn.CrossEntropyLoss()
            scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5, verbose=True)
            
            # Run training and evaluation, now expecting validation loss as the return value
            val_loss = train_and_evaluate_val_loss(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=500, patience=15)  # Adjust num_epochs and patience as needed

            # Update best model tracking based on validation loss
            if val_loss < best_loss:
                best_loss = val_loss
                best_params = {'batch_size': batch_size, 'learning_rate': lr, 'unfreeze_option': unfreeze_option}

print(f"Best Parameters: {best_params}, Best Validation Loss: {best_loss}")
